In [ ]:
# first we'll run through with an example, two versions of OSP
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as nd
import time
import importlib as il
from scipy import interpolate
import netCDF4 as c4

In [ ]:
from hypso import download_nc_files
from hypso import Satellite
            
download_nc_files(filename_list=["balaton_2023-10-03_0921Z-l1a.nc"],
                  output_dir="downl_dir")

In [ ]:
# Import Satellite Object
from hypso import Satellite

# Define HYPSO Image File and the .points from QGIS (if available)
hypso_file_path="downl_dir/balaton_2023-10-03_0921Z-l1a.nc"
#points_path = "/Users/alvaroflores/Documents/mjosa_2023-06-15_0948Z-rgba_8bit.tif.points"

# Create Satellite Object
satobj = Satellite(hypso_file_path)

In [ ]:
data = satobj.l1b_cube
height = data.shape[0]
width = data.shape[1]
bands = data.shape[2]
print("Height: ", height, " Width: ", width, "Bands: ", bands)

In [ ]:
rgb = np.rot90(satobj.l1b_cube[:,:,[70,45,20]])
for i in range(3):
   rgb[:,:,i] -= rgb[:,:,i].min()
plt.imshow(rgb/30, aspect=0.3)

In [ ]:
# setting the path to where i've stored the dbn file

In [ ]:
#cd DimensionalityReduction/

In [ ]:
import dbn as dbn

In [ ]:
aen = dbn.DBN(Verbose=True)

In [ ]:
start = time.time()
aen.fit(data.reshape((-1,bands)))
delta = time.time() - start

In [ ]:
aent = aen.transform(data.reshape((-1,bands)))

In [ ]:
# display some of the transformed data
fig, ax = plt.subplots(1,4)
ax[0].imshow(aent[0].reshape((height, width)), aspect=3)
ax[1].imshow(aent[2].reshape((height, width)), aspect=3)
ax[2].imshow(aent[4].reshape((height, width)), aspect=3)
ax[3].imshow(aent[11].reshape((height, width)), aspect=3)
for a in ax:
    a.set_xticks([])
    a.set_yticks([])

In [ ]:
aen_est = aen.inverse_transform(aent)

In [ ]:
aen_est.shape

In [ ]:
data.reshape((-1,bands))

In [ ]:
aen_est

In [ ]:
# find the error
err = np.sqrt(np.sum((data.reshape((-1,bands)) - aen_est)**2, axis=-1))

# find the spectral angle between the reconstruction and the original pixel
d_mag = np.sqrt(np.sum((np.float32(data.reshape((-1,bands))))**2, axis=-1))
ae_mag = np.sqrt(np.sum((aen_est**2), axis=-1))
sam = ((np.sum(np.multiply(data.reshape((-1,bands)), aen_est), axis=-1))/(d_mag*ae_mag))

In [ ]:
# show anamolous pixels based on error
plt.imshow((err/d_mag).reshape((height, width)), interpolation='nearest', vmax=(err/d_mag).max()/5)
plt.colorbar()

In [ ]:
# show anamolous pixels based on SAM
plt.imshow(np.arccos((sam)).reshape((height,width)), interpolation='nearest', vmax=sam.max()/10)
plt.colorbar()

In [ ]:
import dsw as dsw
import time

con = {}
con['height'] = height
con['width'] = width
con['win_dif'] = 3
con['win_size'] = 15 # MUST BE ODD
con['code_size'] = 12
#anomaly_scores = dsw.novel_dsw(con, err, np.transpose(aent))
anomaly_scores_opt = dsw.opt_novel_dsw(con, err, np.transpose(aent))
# Print the results
#print(anomaly_scores)
print(anomaly_scores_opt)


In [ ]:
plt.imshow(anomaly_scores_opt, interpolation='nearest', vmax=anomaly_scores_opt.max()/40)
plt.colorbar()
#print(anomaly_scores_opt)